In [29]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [30]:
print(os.environ["PATH"])

C:\Users\ravit\Miniconda3\envs\dv;C:\Users\ravit\Miniconda3\envs\dv\Library\mingw-w64\bin;C:\Users\ravit\Miniconda3\envs\dv\Library\usr\bin;C:\Users\ravit\Miniconda3\envs\dv\Library\bin;C:\Users\ravit\Miniconda3\envs\dv\Scripts;C:\Users\ravit\Miniconda3\envs\dv\bin;C:\Users\ravit\Miniconda3\Library\bin;C:\Users\ravit\Miniconda3;C:\Users\ravit\Miniconda3\Library\mingw-w64\bin;C:\Users\ravit\Miniconda3\Library\usr\bin;C:\Users\ravit\Miniconda3\Library\bin;C:\Users\ravit\Miniconda3\Scripts;C:\Users\ravit\Miniconda3\Library\bin;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\iCLS\;C:\Program Files\Intel\Intel(R) Management Engine Components\iCLS\;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files (x86)\Intel\Intel(R)

In [31]:
from sklearn import tree
from dtreeviz.trees import *
import pandas as pd
from sklearn.utils import resample
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn import svm
import sklearn.metrics as sm
import numpy as np


def get_label(row):
    if "<=50K" in row.income:
        return 0
    return 1

adult_df = pd.read_csv("C:/Users/ravit/Project/adult.csv")
adult_df = adult_df.fillna(0)
adult_df['label'] = adult_df.apply(lambda row: get_label(row), axis=1)
adult_df.drop(columns=['income', 'fnlwgt', 'education'],inplace=True)

adult_df['workclass'], val1 = pd.factorize(adult_df['workclass'])
# adult_df['education'] = preprocessing.LabelEncoder().fit_transform(adult_df['education'])
adult_df['marital-status'], val2 = pd.factorize(adult_df['marital-status'])
adult_df['native-country'], val0 = pd.factorize(adult_df['native-country'])
adult_df['occupation'], val3 = pd.factorize(adult_df['occupation'])
adult_df['relationship'], val4 = pd.factorize(adult_df['relationship'])
adult_df['race'], val5 = pd.factorize(adult_df['race'])
adult_df['sex'], val6 = pd.factorize(adult_df['sex'])

adult_df.to_csv("factorized_file.csv", sep=',', encoding='utf-8')
# adult_df.head()
label0_df = adult_df[adult_df.label == 0]
label1_df = adult_df[adult_df.label == 1]


df_minority_upsampled = resample(label1_df, replace=True, n_samples = label0_df.shape[0], random_state=123)

result_df = pd.concat([label0_df, df_minority_upsampled])

msk = np.random.rand(len(result_df)) < 0.8
train_df = result_df[msk]
test_df = result_df[~msk]
df_upsampled = train_df.copy()

y = df_upsampled.label
X = df_upsampled.drop('label', axis=1)
feature_list = list(result_df)
print(feature_list)

y_test=test_df.label
x_test=test_df.drop('label', axis=1)

# model = svm.SVC(kernel='linear', class_weight='balanced', C=1, gamma=1)
model= tree.DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
# scaler = StandardScaler()

# X_std = scaler.fit_transform(X)
# x_test_std = scaler.fit_transform(x_test)

model.fit(X, y)
predicted= model.predict(x_test)

print('training score',model.score(X, y))
print('testing score',sm.accuracy_score(y_test,predicted))
print('f1 score',sm.f1_score(y_test,predicted))
print('recall',sm.recall_score(y_test,predicted))
print('precision',sm.precision_score(y_test,predicted))

['age', 'workclass', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']
training score 0.7994233833236387
testing score 0.8042226487523992
f1 score 0.817204301075269
recall 0.87409200968523
precision 0.767268862911796


In [ ]:
tree.export_graphviz(model, out_file='tree.dot', feature_names=feature_list[:len(feature_list)-1])
Source.from_file('tree.dot')
viz = dtreeviz(model, X, y, target_name='Income', feature_names=feature_list[:len(feature_list)-1], 
               class_names=["0", "1"])  # need class_names for classifier)  
              
viz.view()